# Network Deliberation ABM
## Plot discrete state, NK Model task

In [ ]:
# Imports
import configparser
import math
import random

import matplotlib
from matplotlib import pyplot as plt
import networkx as nx
import nkmodel as nkmodel
import numpy as np
from numpy import random as nprand
import repsci
import scipy.stats as spstats
from tqdm.notebook import tqdm

import netdelib.soclearn as slearn
import netdelib.soclearn.evaluate as sleval
import netdelib.soclearn.models.generated as slgen
import netdelib.soclearn.strategy as slstrat
from netdelib.topologies.factories import *

from discrete import run_discrete

%pylab inline

mpl.rcParams['font.size'] = 7
mpl.rcParams['figure.dpi'] = 600
mpl.rcParams['lines.markersize'] = 7

In [ ]:
timestamp = "2022-07-07 135221 59d6c3b"
experiment = repsci.Experiment("discrete_generated_nk", reproduce=timestamp)

# Configuration
config = experiment.get_config()

N = config.getint('abm', 'N')
M = config.getint('abm', 'M')
runs = config.getint('abm', 'runs')
stages = config.getint('abm', 'stages')
steps = config.getint('abm', 'steps')

bit_count = config.getint('abm', 'bit_count')
K = config.getint('abm', 'K')
p_error = config.getfloat('abm', 'p_error')
nk_exponent = config.getfloat('abm', 'nk_exponent')

small_world_k = config.getint('networks', 'small_world_k')
small_world_a = config.getfloat('networks', 'small_world_a')
barabasi_albert_m = config.getint('networks', 'barabasi_albert_m')
er_p = config.getfloat('networks', 'erdos_renyi_p')

all_networks = {
    'Complete': CompleteFactory(N, M),
    'Lattice': SmallWorldFactory(N, M, small_world_k, 0),
    'Pref. Attach.': PreferentialFactory(N, M, barabasi_albert_m),
    'Small World': SmallWorldFactory(N, M, small_world_k, small_world_a),
    'LFR': LFRFactory(N, M, 1.5, 1.5, 1/4),
    'StochasticBlock': StochasticBlockFactory(N, M),
    'Random': RandomFactory(N, M, er_p),
    'Long Path': LongPathFactory(N, M),
    'Random Group': RandomGroupFactory(N, M),
}
networks = [
    (k, all_networks[k])
    for k in config.get('networks', 'enabled').split(',')]

# Configure Strategies
individual = config.getboolean('strategies', 'individual')
individual_mode = config.getint('strategies', 'individual_mode')
individual_all_bits = config.getboolean('strategies', 'individual_all_bits')
critical = config.getboolean('strategies', 'critical')
sample = config.getint('strategies', 'sample', fallback=None)
all_learning_strategies = {
    'Best Neighbor': slstrat.best_neighbor,
    'Confident Neighbor': slstrat.confident_neighbor,
    'Conform': slstrat.conform,
    'Local Majority': slstrat.local_majority,
}
learning_strategies = [
    (k, all_learning_strategies[k])
    for k in config.get('strategies', 'enabled').split(',')
]

num_net = len(networks)
num_strat = len(learning_strategies)

In [ ]:
file = experiment.get_filename('score.npy')
score = np.load(file)
file = experiment.get_filename('correct.npy')
correct = np.load(file)
file = experiment.get_filename('consensus.npy')
consensus = np.load(file)
file = experiment.get_filename('basin.npy')
basin = np.load(file)
file = experiment.get_filename('basin_count.npy')
basin_count = np.load(file)
file = experiment.get_filename('distance_score.npy')
distance_score = np.load(file)
file = experiment.get_filename('initial_distance.npy')
initial_distance = np.load(file)
file = experiment.get_filename('converge_time.npy')
converge_time = np.load(file)
file = experiment.get_filename('transition_higher.npy')
transition_higher = np.load(file)
file = experiment.get_filename('transition_lower.npy')
transition_lower = np.load(file)
file = experiment.get_filename('higher_possible.npy')
higher_possible = np.load(file)

In [ ]:
file = experiment.get_filename('score_run.npy')
score_for_strat_net_run = np.load(file)
file = experiment.get_filename('correct_run.npy')
correct_for_strat_net_run = np.load(file)
file = experiment.get_filename('consensus_run.npy')
consensus_for_strat_net_run = np.load(file)
file = experiment.get_filename('basin_run.npy')
basin_for_strat_net_run = np.load(file)
file = experiment.get_filename('basin_count_run.npy')
basin_count_for_strat_net_run = np.load(file)
file = experiment.get_filename('distance_score_run.npy')
distance_score_for_strat_net_run = np.load(file)
file = experiment.get_filename('initial_distance_run.npy')
initial_distance_for_strat_net_run = np.load(file)
file = experiment.get_filename('converge_time_run.npy')
converge_time_for_strat_net_run = np.load(file)
file = experiment.get_filename('transition_higher_run.npy')
transition_higher_for_strat_net_run = np.load(file)
file = experiment.get_filename('transition_lower_run.npy')
transition_lower_for_strat_net_run = np.load(file)
file = experiment.get_filename('higher_possible_run.npy')
higher_possible_for_strat_net_run = np.load(file)

In [ ]:
plt.figure(figsize=(16, 12))
for i, (title, learning_strategy) in enumerate(learning_strategies):
    for j, (title2, factory) in enumerate(networks):
        plt.subplot(num_strat, num_net, 1 + i * num_net + j)

        y = score[i][j][0]
        x = range(len(y))
        err95 = 1.96 * np.array(score[i][j][1]) / math.sqrt(runs)
        plt.fill_between(x, y - err95, y + err95, color="#aaaaff7f")
        plt.plot(x, y, 'k-')     

        y = correct[i][j][0]
        x = range(len(y))
        err95 = 1.96 * np.array(correct[i][j][1]) / math.sqrt(runs)
        plt.fill_between(x, y - err95, y + err95, color="#aaaaff7f")
        plt.plot(x, y, 'b:')
        
        y = distance_score[i][j][0]
        x = range(len(y))
        err95 = 1.96 * np.array(distance_score[i][j][1]) / math.sqrt(runs)
        plt.fill_between(x, y - err95, y + err95, color="#aaaaff7f")
        plt.plot(x, y, 'g--')
        
        plt.title(f'{title} - {title2}')
        if j == 0:
            plt.ylabel('Frac. Correct')
        if i == 2:
            plt.xlabel('Time Step')
        plt.grid(True)
        plt.ylim([0.0, 1])

out = experiment.get_filename('plot.png')
if individual:
    if individual_all_bits:
        title_individual = 'all-bit'
    else:
        title_individual = 'single-bit'
    if individual_mode == slearn.MODE_ALL:
        title_individual += ' every-step individual'
    elif individual_mode == slearn.MODE_FALLBACK:
        title_individual += ' fal  lback-only individual'
    elif individual_mode == slearn.MODE_BEST:
        title_individual += ' best individual'
else:
    title_individual = 'social-only'
title = "N={}, K={}, EXP={}, S={}, {}, {}".format(
    bit_count,
    K,
    nk_exponent,
    sample,
    {True: 'critical', False: 'non-critical'}[critical],
    title_individual)
plt.suptitle(title)
plt.savefig(out, dpi=300)

In [ ]:
plt.figure(figsize=(16, 12))
for i, (title, learning_strategy) in enumerate(learning_strategies):
    for j, (title2, factory) in enumerate(networks):
        plt.subplot(num_strat, num_net, 1 + i * num_net + j)

        y = score[i][j][0]
        x = range(len(y))
        err95 = 1.96 * np.array(score[i][j][1]) / math.sqrt(runs)
        plt.fill_between(x, y - err95, y + err95, color="#aaaaff7f")
        plt.plot(x, y, 'k-')     

        y = consensus[i][j][0]
        x = range(len(y))
        err95 = 1.96 * np.array(consensus[i][j][1]) / math.sqrt(runs)
        plt.fill_between(x, y - err95, y + err95, color="#aaaaff7f")
        plt.plot(x, y, 'b:')
        
        plt.title(f'{title} - {title2}')
        if j == 0:
            plt.ylabel('Frac. Consensus')
        if i == 2:
            plt.xlabel('Time Step')
        plt.grid(True)
        plt.ylim([0.0, 1])

out = experiment.get_filename('plot-consensus.png')
if individual:
    if individual_all_bits:
        title_individual = 'all-bit'
    else:
        title_individual = 'single-bit'
    if individual_mode == slearn.MODE_ALL:
        title_individual += ' every-step individual'
    elif individual_mode == slearn.MODE_FALLBACK:
        title_individual += ' fal  lback-only individual'
    elif individual_mode == slearn.MODE_BEST:
        title_individual += ' best individual'
else:
    title_individual = 'social-only'
title = "N={}, K={}, EXP={}, S={}, {}, {}".format(
    bit_count,
    K,
    nk_exponent,
    sample,
    {True: 'critical', False: 'non-critical'}[critical],
    title_individual)
plt.suptitle(title)
plt.savefig(out, dpi=300)

In [ ]:
plt.figure(figsize=(16, 12))
for i, (title, learning_strategy) in enumerate(learning_strategies):
    for j, (title2, factory) in enumerate(networks):
        plt.subplot(num_strat, num_net, 1 + i * num_net + j)

        y = score[i][j][0]
        x = range(len(y))
        err95 = 1.96 * np.array(score[i][j][1]) / math.sqrt(runs)
        plt.fill_between(x, y - err95, y + err95, color="#aaaaff7f")
        plt.plot(x, y, 'k-')     

        y = basin[i][j][0]
        x = range(len(y))
        err95 = 1.96 * np.array(basin[i][j][1]) / math.sqrt(runs)
        plt.fill_between(x, y - err95, y + err95, color="#aaaaff7f")
        plt.plot(x, y, 'b:')
        
        plt.title(f'{title} - {title2}')
        if j == 0:
            plt.ylabel('Frac. Consensus Basin')
        if i == 2:
            plt.xlabel('Time Step')
        plt.grid(True)
        plt.ylim([0.0, 1])

out = experiment.get_filename('plot-consensus-basin.png')
if individual:
    if individual_all_bits:
        title_individual = 'all-bit'
    else:
        title_individual = 'single-bit'
    if individual_mode == slearn.MODE_ALL:
        title_individual += ' every-step individual'
    elif individual_mode == slearn.MODE_FALLBACK:
        title_individual += ' fal  lback-only individual'
    elif individual_mode == slearn.MODE_BEST:
        title_individual += ' best individual'
else:
    title_individual = 'social-only'
title = "N={}, K={}, EXP={}, S={}, {}, {}".format(
    bit_count,
    K,
    nk_exponent,
    sample,
    {True: 'critical', False: 'non-critical'}[critical],
    title_individual)
plt.suptitle(title)
plt.savefig(out, dpi=300)

In [ ]:
labels = {
    'Pref. Attach.': 'Pref.-Attach.',
    'Random Group': 'ND Random-Pod',
    'Long Path': 'ND Long-Path',
    'Small World': 'Small-World'
}

plt.figure(figsize=(3, 4))
for i, (title, learning_strategy) in enumerate(learning_strategies):
    if title != 'Conform':
        continue
    for j, (title2, factory) in enumerate(networks):
        ax = plt.subplot(4, 1, 1 + j)

        # We skip the first because values are only meaningful after social learning has run
        y = transition_higher[i][j][0][1:]
        x = range(1, len(y) + 1)
        err95 = 1.96 * np.array(transition_higher[i][j][1][1:]) / math.sqrt(runs)
        plt.fill_between(x, y - err95, y + err95, color="#aaaaff7f")
        plt.plot(x, y, 'b--', label='Higher Basin')
        
        y = transition_lower[i][j][0][1:]
        x = range(1, len(y) + 1)
        err95 = 1.96 * np.array(transition_lower[i][j][1][1:]) / math.sqrt(runs)
        plt.fill_between(x, y - err95, y + err95, color="#aaaaff7f")
        plt.plot(x, y, 'r:', label='Lower Basin')

        plt.ylabel(labels[title2])
        if j == 3:
            plt.xlabel('Time Step')
        plt.grid(True)
        plt.ylim([0,30])
        plt.xlim([0,30])
        plt.legend(fontsize=5)

if individual:
    if individual_all_bits:
        title_individual = 'all-bit'
    else:
        title_individual = 'single-bit'
    if individual_mode == slearn.MODE_ALL:
        title_individual += ' every-step individual'
    elif individual_mode == slearn.MODE_FALLBACK:
        title_individual += ' fal  lback-only individual'
    elif individual_mode == slearn.MODE_BEST:
        title_individual += ' best individual'
else:
    title_individual = 'social-only'
plt.tight_layout()
out = experiment.get_filename('fig-conform-transition-basin.eps')
plt.savefig(out)
out = experiment.get_filename('fig-conform-transition-basin.png')
plt.savefig(out, dpi=600)

In [ ]:
fmt = {
    'Pref. Attach.': '-.',
    'Small World': '-.',
    'Long Path': '--',
    'Random Group': '--'
}

color = {
    'Pref. Attach.': '#aa0',
    'Small World': '#660',
    'Long Path': '#a0a',
    'Random Group': '#606'
}

net = {
    'Pref. Attach.': 'Pref. Attach.',
    'Small World': 'Small-World',
    'Long Path': 'ND Long-Path',
    'Random Group': 'ND Random-Pod'
}

fig = plt.figure(figsize=(3, 1.5))
ax = fig.add_subplot(1,1,1)
for i, (title, learning_strategy) in enumerate(learning_strategies):
    if title != 'Conform':
        continue
    for j, (title2, factory) in enumerate(networks):

        y = basin_count[i][j][0]
        x = range(0, len(y))
        err95 = 1.96 * np.array(basin_count[i][j][1]) / math.sqrt(runs)
        plt.fill_between(x, y - err95, y + err95, color="#aaaaff7f")
        plt.plot(x, y, fmt[title2], label=net[title2], color=color[title2])
        
        plt.ylabel('Occupied Basins')
        plt.xlabel('Time Step')
        if i == 4:
            plt.xlabel('Time Step')
        plt.grid(True)
        plt.xlim([0,30])
        plt.ylim([0,100])
        
import copy
handles, labels = ax.get_legend_handles_labels()
handles = [copy.copy(ha) for ha in handles ]
[ha.set_linewidth(1) for ha in handles ]
leg = plt.legend(handles=handles, labels=labels, fontsize=5)

if individual:
    if individual_all_bits:
        title_individual = 'all-bit'
    else:
        title_individual = 'single-bit'
    if individual_mode == slearn.MODE_ALL:
        title_individual += ' every-step individual'
    elif individual_mode == slearn.MODE_FALLBACK:
        title_individual += ' fal  lback-only individual'
    elif individual_mode == slearn.MODE_BEST:
        title_individual += ' best individual'
else:
    title_individual = 'social-only'
plt.tight_layout()
out = experiment.get_filename('fig-conform-basin.eps')
plt.savefig(out)
out = experiment.get_filename('fig-conform-basin.png')
plt.savefig(out, dpi=600)